# TP Final NLP

## Primero nos encargaremos de los datos. Contamos con dos archivos de texto y un csv, además para base de datos de grafos usaremos wikidata.

In [ ]:
pip install PyPDF2

In [ ]:
!pip install unidecode

In [ ]:
!pip install nltk

In [ ]:
!pip install langid

### Procesar y meter textos en chromaDB

Contamos con 2 textos en pdf, correspondientes a los textos:
"TITANIC HISTORIAS PARA DESPUES DE UN NAUFRAGIO" de Fernando José García Echegoyen.
"OTROS ASPECTOS TÉCNICOS Y HUMANOS DEL RMS “TITANIC”" del Ing. Mec. Sergio Kuczynski

In [ ]:
path_historia = "Titanic Historico.pdf"
path_historia_2 = "Titanic Historico 2.pdf"

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Extraer texto de los PDFs
text_historia = extract_text_from_pdf(path_historia)
text_historia2 = extract_text_from_pdf(path_historia_2)

In [ ]:
#Pasamos texto a minúsculas:
texto_historia_min = text_historia.lower()
texto_historia2_min = text_historia2.lower()

In [ ]:
# Sacamos acentos
from unidecode import unidecode

historia_no_acentos = unidecode(texto_historia_min)
historia2_no_acentos = unidecode(texto_historia2_min)

In [ ]:
# Eliminamos caracteres especiales
import re

sin_especiales_historia = re.sub(r'[^a-zA-Z0-9\s]', '', historia_no_acentos)
sin_especiales_historia2 = re.sub(r'[^a-zA-Z0-9\s]', '', historia2_no_acentos)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Descargar el tokenizador de palabras de NLTK si no está descargado
nltk.download('punkt')

# Función para dividir una lista de tokens en fragmentos cada 500 palabras
def split_tokens_by_word_count(tokens, word_count=500):
    fragments = []
    current_fragment_words = []

    for word in tokens:
        current_fragment_words.append(word)

        if len(current_fragment_words) >= word_count:
            fragments.append(current_fragment_words)
            current_fragment_words = []

    if current_fragment_words:
        fragments.append(current_fragment_words)

    return fragments

# Tokenizar los textos
tokens_historia = word_tokenize(sin_especiales_historia)
tokens_historia2 = word_tokenize(sin_especiales_historia2)

# Dividir los tokens en fragmentos cada 500 palabras
fragmentos_historia = split_tokens_by_word_count(tokens_historia, word_count=500)
fragmentos_historia2 = split_tokens_by_word_count(tokens_historia2, word_count=500)

# Imprimir los fragmentos resultantes
print("Fragmentos de la novela:")
for i, fragmento in enumerate(fragmentos_historia):
    print(f"Fragmento {i + 1}: {fragmento}")

print("\nFragmentos del guion:")
for i, fragmento in enumerate(fragmentos_historia2):
    print(f"Fragmento {i + 1}: {fragmento}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fragmentos de la novela:
Fragmento 1: ['1', 'titanic', 'historias', 'para', 'despues', 'de', 'un', 'naufragio', 'fernando', 'jose', 'garcia', 'echegoyen', '2publicaciones', 'de', 'wwwnaufragioses', 'copyright', 'fernando', 'jose', 'garcia', 'echegoyen', '2012', 'prohibida', 'la', 'reproduccion', 'sin', 'el', 'permiso', 'por', 'escrito', 'del', 'autor', 'esta', 'edicion', 'de', 'titanic', 'historias', 'para', 'despues', 'de', 'un', 'naufragio', 'es', 'una', 'edicion', 'digital', 'y', 'no', 'venal', 'se', 'puede', 'obtener', 'de', 'forma', 'gratuita', 'mediante', 'descarga', 'en', 'la', 'web', 'wwwnaufragioses', 'y', 'se', 'edita', 'para', 'conmemorar', 'el', 'centenario', 'del', 'hundimiento', 'del', 'titanic', 'dedicado', 'a', 'la', 'memoria', 'de', 'don', 'jose', 'luis', 'rodriguez', 'carrion', 'capitan', 'de', 'la', 'marina', 'mercante', 'doctor', 'en', 'derecho', 'y', 'catedratico', 'de', 'derecho', 'maritimo', 'de', 'la', 'escuela', 'superior', 'de', 'la', 'marina', 'civil', 'de', 

In [ ]:
# Convertir cada fragmento de tokens en una cadena continua
fragmentos_historia = [' '.join(fragmento) for fragmento in fragmentos_historia]
fragmentos_historia2 = [' '.join(fragmento) for fragmento in fragmentos_historia2]

# Imprimir los fragmentos resultantes
print("Fragmentos de la historia:")
for i, fragmento in enumerate(fragmentos_historia):
    print(f"Fragmento {i + 1}: {fragmento}")

print("\nFragmentos del historia2:")
for i, fragmento in enumerate(fragmentos_historia2):
    print(f"Fragmento {i + 1}: {fragmento}")


Fragmentos de la historia:
Fragmento 1: 1 titanic historias para despues de un naufragio fernando jose garcia echegoyen 2publicaciones de wwwnaufragioses copyright fernando jose garcia echegoyen 2012 prohibida la reproduccion sin el permiso por escrito del autor esta edicion de titanic historias para despues de un naufragio es una edicion digital y no venal se puede obtener de forma gratuita mediante descarga en la web wwwnaufragioses y se edita para conmemorar el centenario del hundimiento del titanic dedicado a la memoria de don jose luis rodriguez carrion capitan de la marina mercante doctor en derecho y catedratico de derecho maritimo de la escuela superior de la marina civil de cadiz quien hace treinta anos dirigio mi trabajo sobre el hundimiento del titanic en el cual esta basado el presente libro 3indice introduccion 6 primera parte historias para despues de un naufragio 11 y en la senda de los elefantes los cohetes iluminaron la noche 10 12 los barcos funebres 21 13 la investig

Compruebo la cantidad de fragmentos para después comprobar que estoy obteniendo el número correctode embeddings

In [ ]:
len(fragmentos_historia)

99

In [ ]:
len(fragmentos_historia2)

247

In [ ]:
#Eliminación de stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

# Definir la función para eliminar stopwords
def eliminar_stopwords(tokens):
    stop_words = set(stopwords.words('spanish'))
    return [word for word in tokens if word.lower() not in stop_words]

# Aplicar la eliminación de stopwords a cada conjunto de tokens
tokens_historia_sin_stopwords = eliminar_stopwords(fragmentos_historia)
tokens_historia2_sin_stopwords = eliminar_stopwords(fragmentos_historia2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


A partir de este punto "fragmentos_..." corresponderán a la versión sin stopwords, que será la versión de la cual obtendremos los embeddings

In [ ]:
fragmentos_historia = tokens_historia_sin_stopwords
fragmentos_historia2 = tokens_historia2_sin_stopwords

Usaremos BERT para obtener los embeddings

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

# Cargar el modelo BERT en español
model = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Función que obtiene embeddings para cada texto
def obtener_embeddings(fragmentos):
    embeddings = []
    for fragmento in fragmentos:
        tokens = tokenizer(fragmento, truncation=True, padding=True, return_tensors='pt', max_length=512)
        outputs = model(**tokens)
        embedding_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
        embeddings.append(embedding_vector.tolist())
    return embeddings

# Obtener embeddings para cada fragmento
embedding_historia = obtener_embeddings(fragmentos_historia)
embedding_historia2 = obtener_embeddings(fragmentos_historia2)

# Imprimir los resultados
print("Embeddings de los fragmentos de historia:", np.array(embedding_historia))
print("Embeddings de los fragmentos de historia2:", np.array(embedding_historia2))

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embeddings de los fragmentos de historia: [[-0.51368707 -0.03130415 -0.48294505 ... -0.11937986 -0.10678983
   0.23667356]
 [-0.47404432 -0.02554024 -0.36945891 ... -0.24433538 -0.04726356
   0.15014389]
 [-0.37117237  0.12446852 -0.42044544 ... -0.23932049 -0.04399152
   0.07126554]
 ...
 [-0.22148106 -0.01882081 -0.44791812 ... -0.10493489 -0.24792856
   0.08321685]
 [-0.57851946 -0.08186018 -0.41757947 ... -0.08008375 -0.22635603
   0.17111251]
 [-0.47379166 -0.14672618 -0.4341948  ... -0.20593043 -0.12248923
   0.1162861 ]]
Embeddings de los fragmentos de historia2: [[-0.42105639  0.01075189 -0.47865167 ... -0.05775309  0.13322982
   0.27421394]
 [-0.42463672 -0.0210679  -0.42561054 ... -0.02754493  0.07646979
   0.18072405]
 [-0.35788089 -0.03703925 -0.41301063 ... -0.09740598  0.1536614
   0.19261748]
 ...
 [-0.30516472 -0.06484303 -0.4121471  ... -0.05457179 -0.1076207
   0.0569527 ]
 [-0.28740582  0.01013252 -0.35456401 ...  0.02527983 -0.11425823
   0.08675576]
 [-0.38623673 -

Comprobamos que tengamos el número correcto de embeddings para cada grupo de fragmentos

In [ ]:
print(len(embedding_historia))
print(len(embedding_historia2))

99
247


#### Guardar los embeddings en ChromaDB

In [ ]:
#Seguimos los pasos de la página de ChromaDB
!pip install chromadb

  Using cached fastapi-0.109.0-py3-none-any.whl (92 kB)
  Using cached starlette-0.35.1-py3-none-any.whl (71 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.23.1
    Uninstalling starlette-0.23.1:
      Successfully uninstalled starlette-0.23.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.90.1
    Uninstalling fastapi-0.90.1:
      Successfully uninstalled fastapi-0.90.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [ ]:
#Corrigiendo el error del siguiente paso, con la solución propuesta por la IA de Colab
!pip install fastapi==0.90.1

  Using cached fastapi-0.90.1-py3-none-any.whl (56 kB)
  Using cached starlette-0.23.1-py3-none-any.whl (64 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.35.1
    Uninstalling starlette-0.35.1:
      Successfully uninstalled starlette-0.35.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.109.0
    Uninstalling fastapi-0.109.0:
      Successfully uninstalled fastapi-0.109.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
chromadb 0.4.22 requires fastapi>=0.95.2, but you have fastapi 0.90.1 which is incompatible.


In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.get_collection(name="datos_titanic")
if collection is not None:
    print("Collection already exists.")
else:
    # Create the collection if it doesn't exist
    collection = chroma_client.create_collection(name="datos_titanic")

Collection already exists.


Ahora unimos los fragmentos de texto en una sola lista para que sea más sencillo agregarlos a chromaDB

In [ ]:
fragmentos_titanic = []
# Extender la lista fragmentos_titanic con los fragmentos de fragmentos_historia y fragmentos_historia2
fragmentos_titanic.extend(fragmentos_historia)
fragmentos_titanic.extend(fragmentos_historia2)

Comprobamos que la nueva lista tenga el número correcto de fragmentos, debería ser la suma de las anteriores, es decir 346

In [ ]:
len(fragmentos_titanic)

346

Realizamos el mismo proceso para los embeddings

In [ ]:
embeddings_titanic = []

embeddings_titanic.extend(embedding_historia)
embeddings_titanic.extend(embedding_historia2)

In [ ]:
len(embeddings_titanic)

346

Generamos Id´s para los fragmentos ya que chromaDB nos pide que cada fragmento tenga su Id

In [ ]:
# Crear IDs
ids_titanic = [f'id{i+1}' for i in range(len(fragmentos_titanic))]

In [ ]:
#Agregamos los embeddings a chromaDB
collection.add(
    embeddings=embeddings_titanic,
    documents=fragmentos_titanic,
    ids=ids_titanic
)

### Procesar csv

In [ ]:
import pandas as pd

In [ ]:
path_csv = 'Titanic.csv'

In [ ]:
# Cargar el archivo CSV en un DataFrame de pandas
df_titanic = pd.read_csv(path_csv)

In [ ]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Acá vamos a pasar el dataframe a un string el cual usaremos como respuesta en caso de necesitar información del csv.

In [ ]:
def dataframe_to_string(df):
    result = ""
    for index, row in df.iterrows():
        # Reemplazamos los valores de "Survived" por "no" si es 0, o por "si" si es 1
        survived = "no" if row['Survived'] == 0 else "si"

        result += f"PassengerId: {row['PassengerId']}\n"
        result += f"Sobrevivio: {survived}\n"
        result += f"Clase: {row['Pclass']}\n"
        result += f"Nombre: {row['Name']}\n"
        result += f"Sexo: {row['Sex']}\n"
        result += f"Edad: {row['Age']}\n"
        result += f"Parch: {row['Parch']}\n"
        result += f"Boleto: {row['Ticket']}\n"
        result += f"Precio: {row['Fare']}\n"
        result += f"Cabina: {row['Cabin']}\n"
        result += f"Embarco: {row['Embarked']}\n"
        result += "\n"

    return result

# Convertimos el DataFrame en un string
result_string = dataframe_to_string(df_titanic)

# Imprimimos el resultado
print(result_string)


PassengerId: 892
Sobrevivio: no
Clase: 3
Nombre: Kelly, Mr. James
Sexo: male
Edad: 34.5
Parch: 0
Boleto: 330911
Precio: 7.8292
Cabina: nan
Embarco: Q

PassengerId: 893
Sobrevivio: si
Clase: 3
Nombre: Wilkes, Mrs. James (Ellen Needs)
Sexo: female
Edad: 47.0
Parch: 0
Boleto: 363272
Precio: 7.0
Cabina: nan
Embarco: S

PassengerId: 894
Sobrevivio: no
Clase: 2
Nombre: Myles, Mr. Thomas Francis
Sexo: male
Edad: 62.0
Parch: 0
Boleto: 240276
Precio: 9.6875
Cabina: nan
Embarco: Q

PassengerId: 895
Sobrevivio: no
Clase: 3
Nombre: Wirz, Mr. Albert
Sexo: male
Edad: 27.0
Parch: 0
Boleto: 315154
Precio: 8.6625
Cabina: nan
Embarco: S

PassengerId: 896
Sobrevivio: si
Clase: 3
Nombre: Hirvonen, Mrs. Alexander (Helga E Lindqvist)
Sexo: female
Edad: 22.0
Parch: 1
Boleto: 3101298
Precio: 12.2875
Cabina: nan
Embarco: S

PassengerId: 897
Sobrevivio: no
Clase: 3
Nombre: Svensson, Mr. Johan Cervin
Sexo: male
Edad: 14.0
Parch: 0
Boleto: 7538
Precio: 9.225
Cabina: nan
Embarco: S

PassengerId: 898
Sobrevivio: si

### Probando la info de chromaDB


Llamamos a la colección que creamos e intentamos obtener un contexto para nuestra pregunta, comprobando que efectivamente el texto fue subido a chromaDB.

In [ ]:
# Example query
query_texts = "¿Desde dónde partió el Titanic?"
query_embedding = obtener_embeddings(query_texts)
# print(query_embedding)
results = collection.query(query_embeddings= query_embedding, n_results=5)
for result in results['documents'][0]:
    print(result)
    print('\n')

que dan origen a la mi sma asi a vuelo de pajaro se ndria lo siguiente mitologia griega esis anti uo te libro del gen g testamento prometeo dios yahve prometeo poseedor del fuego del cielo dios poseedor del espiritu d vida e hombre de barr creado por prometeo adan creado por dios utilizando arcilla o prometeo roba el fuego del cielo dios concede el espiritu de vida a adan jupiter deidad superior a prometeo yahve divin idad suprema ordena al angel del senor ordena a vulcano encadenar a prometeo que adan y eva no vuelvan al ja eden rdin del pena temporal de cumplimiento efecti vo pena de eterno cumplimiento treinta anos acion de la pena ena hast a la mu te fisica duracion dur la vida terr er de la pena hercules liber tador de prometeo la muerte eterna mediante dios libera al humano de la promesa de jesucristo vulcano verdugo de prometeo y creador de pand ora espiritu de dios juez de adan y eva pandora eva jupiter deidad superior a prometeo te del cuerpo de adan dios utiliza una par orden

### Conectando con Wikidata
Para nuestra base de datos de grafos, utilizaremos wikidata.

In [ ]:
!pip install wikidataintegrator

In [ ]:
wikidata_user = 'LeoSebaPale'
wikidata_password = 'Sunyday@18'

In [ ]:
# Importar las bibliotecas
from wikidataintegrator import wdi_core, wdi_login

# Solicitar al usuario que ingrese su nombre de usuario y contraseña
wikidata_user = input("Ingresa tu nombre de usuario de Wikidata: ")
wikidata_password = input("Ingresa tu contraseña de Wikidata: ")


Ingresa tu nombre de usuario de Wikidata: LeoSebaPale
Ingresa tu contraseña de Wikidata: Sunyday@18


In [ ]:
def obtener_info_personaje(label_sujeto):
    # Iniciar sesión en Wikidata
    login_instance = wdi_login.WDLogin(wikidata_user, wikidata_password)

    # Probar que la conexión funciona con una consulta SPARQL
    sparql_query = f'''
    SELECT ?sujeto ?sujetoLabel
    WHERE {{
      ?sujeto rdfs:label "{label_sujeto}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    '''

    # Ejecutar la consulta SPARQL
    result = wdi_core.WDItemEngine.execute_sparql_query(sparql_query, as_dataframe=True)

    # Extracción del ID
    try:
        valor_sujeto = result['sujeto'].str.split('/').str[-1]
    except (KeyError, AttributeError, IndexError):
        # Manejar el error y retornar un string vacío en caso de cualquier excepción esperada
        return ''

    # Imprimir el resultado
    sujeto_id = valor_sujeto.values[0]

    description = f'{label_sujeto}  :\n'

    propiedades = ["P21", "P27", "P1559", "P1477", "P735", "P569", "P19", "P570", "P1196", "P509",
                   "P157", "P22", "P25", "P3373", "P3342", "P106", "P108", "P39", "P551", "P463"]

    for propiedad_id in propiedades:
        # Construir la consulta SPARQL para obtener los valores por sujeto y propiedad
        consulta_sparql = f"""
        SELECT ?property ?propertyLabel ?value ?valueLabel
        WHERE {{
          wd:{sujeto_id} wdt:{propiedad_id} ?value.
          ?property wikibase:directClaim wdt:{propiedad_id}.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """

        # URL del punto de consulta SPARQL de Wikidata
        endpoint_url = "https://query.wikidata.org/sparql"

        # Encabezados HTTP necesarios para la consulta a Wikidata
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
            'Accept': 'application/json'
        }

        # Parámetros de la consulta SPARQL
        params = {
            'query': consulta_sparql,
            'format': 'json'
        }

        # Realizar la solicitud HTTP GET
        respuesta = requests.get(endpoint_url, headers=headers, params=params)

        # Verificar si la solicitud fue exitosa (código de estado 200)
        if respuesta.status_code == 200:
            # Convertir la respuesta a formato JSON
            resultados = respuesta.json()

            # Obtener los valores asociados a la propiedad
            valores = [(item['propertyLabel']['value'], item['valueLabel']['value']) for item in resultados['results']['bindings']]

            for propiedad, valor in valores:
                description += f'{propiedad}: {valor} \n'
        else:
            description = ''

    return description

## Eligiendo la fuente correcta y creando las funciones que llamaran al chatbot


In [ ]:
!pip install jinja2

In [ ]:
from jinja2 import Template

In [ ]:
import requests

Usando el código de la unidad 6, lo modificamos agregando un fewshots para que el modelo elija la base de datos que usará para obtener el contexto y responder.

In [ ]:
def zephyr_chat_template(messages, add_generation_prompt=True):
    # Definir la plantilla Jinja
    template_str  = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    # Crear un objeto de plantilla con la cadena de plantilla
    template = Template(template_str)

    # Renderizar la plantilla con los mensajes proporcionados
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)



# Aquí hacemos la llamada al modelo
def elegir_fuente(prompt: str, api_key, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
                                {
                                  "role": "system",
                                  "content": "Sos un asistente bibliográfico, especializado en identificar el sujeto u objeto referido en una pregunta o petición"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Quién fue Edward Smith?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Edward Smith]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuáles fueron las últimas palabras de Jack Dawson?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Jack Dawson]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Qué papel desempeñó Thomas Andrews?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Thomas Andrews]"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuántas personas sobevivieron al choque?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Sobrevivientes"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Cuánto costaron los boletos de quienes viajaron en el Titanic?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Sobrevivientes"
                                },
                                {
                                  "role": "user",
                                  "content": "¿A qué hora partió el Titanic?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Historia"
                                },
                                {
                                  "role": "user",
                                  "content": "¿Desde qué puerto partió el Titanic?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Historia"
                                },
                                {
                                  "role": "user",
                                  "content": f'Responde de una de las siguientes maneras, sin usar información previa del asunto o agregando información que no esté contenida en la pregunta:\n\
                                  "Personaje: [Nombre de un personaje en específico mencionado en la pregunta]" Si es una pregunta o pedido sobre un personaje con el nombre en mayúsculas.\n\
                                  "Sobrevivientes" Si es una pregunta sobre las victimas o sobrevivientes del accidente, de los pasajes o boletos, o de los asientos o cabinas.\n\
                                  "Historia" Si es sobre la historia real del RMS Titanic.\n\
                                  -------------------------------------------\n\
                                  La pregunta o pedido es de la siguiente forma:\n\
                                  {prompt}'
                                }
                              ]


    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        print(respuesta)
        return respuesta


    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import HfApi

In [ ]:
api_key = input("Por favor ingresa tu API key: ")

#### Ejemplo de uso del modelo para elegir la fuente correcta y responder

In [ ]:
# Ejemplo)
prompt = '¿En qué fecha partió el RMS Titanic?'
answer = elegir_fuente(prompt, api_key=api_key)
print('\nRESPUESTA: \n', answer)

Historia: El RMS Titanic zarpó de Southampton, Inglaterra, a las 2:20 p.m. Del jueves 10 de abril de 1912, dirigido al puerto de Nueva York en su primer viaje transatlántico.

RESPUESTA: 
 Historia: El RMS Titanic zarpó de Southampton, Inglaterra, a las 2:20 p.m. Del jueves 10 de abril de 1912, dirigido al puerto de Nueva York en su primer viaje transatlántico.


In [ ]:
import re

Cambiamos el formato de la respuesta para que solo nos entregue lo que nos interesa.

In [ ]:
def formatear_respuesta(texto):
    palabras_clave = ["Personaje", "Sobrevivientes", "Historia"]

    for palabra in palabras_clave:
        if palabra in texto:
            if palabra == "Personaje":
                match = re.search(r'\[([^]]*)\]', texto)
                if match:
                    texto_entre_corchetes = match.group(1)
                    return [palabra, texto_entre_corchetes]
            else:
                return [palabra, None]

    return [None, None]

# Ejemplo de uso:
respuesta_limpia = formatear_respuesta(answer)
print(respuesta_limpia)

['Historia', None]


In [ ]:
from typing import List

Diseñamos la función que elije la base de datos y la llama para obtener la información.

In [ ]:
def devolver_contexto(respuesta_LLM, prompt):
    # Si no se halló nada, no devolver contexto
    if respuesta_LLM in ([None, None], ["Personaje", None]):
        return ("", 'Error al identificar la fuente')

    # Si la pregunta o tarea es referida a un personaje
    elif respuesta_LLM[0] == "Personaje":
        propiedades_personaje = obtener_info_personaje(respuesta_LLM[1])
        return (propiedades_personaje, 'Base de datos de grafos')

    # Si la pregunta o tarea es referida a los sobrevivientes
    elif respuesta_LLM[0] == "Sobrevivientes":
        dataframe_string = dataframe_to_string(df_titanic)
        return (dataframe_string, 'Dataframe')

    # Si la pregunta o tarea es referida a la historia real
    elif respuesta_LLM[0] == "Historia":
        contexto = ''
        # Obtener el cliente Chroma
        chroma_client = chromadb.Client()
        # Obtener la colección
        collection = chroma_client.get_collection(name="datos_titanic")
        # Query
        query_embedding = obtener_embeddings(prompt)
        results = collection.query(query_embeddings= query_embedding, n_results=5)
        for result in results['documents'][0]:
          contexto += result

        return (contexto, "Base de datos vectorial de Embeddings")

In [ ]:
# Ejemplo
contexto = devolver_contexto(respuesta_limpia, prompt)[0]

print(contexto)

que dan origen a la mi sma asi a vuelo de pajaro se ndria lo siguiente mitologia griega esis anti uo te libro del gen g testamento prometeo dios yahve prometeo poseedor del fuego del cielo dios poseedor del espiritu d vida e hombre de barr creado por prometeo adan creado por dios utilizando arcilla o prometeo roba el fuego del cielo dios concede el espiritu de vida a adan jupiter deidad superior a prometeo yahve divin idad suprema ordena al angel del senor ordena a vulcano encadenar a prometeo que adan y eva no vuelvan al ja eden rdin del pena temporal de cumplimiento efecti vo pena de eterno cumplimiento treinta anos acion de la pena ena hast a la mu te fisica duracion dur la vida terr er de la pena hercules liber tador de prometeo la muerte eterna mediante dios libera al humano de la promesa de jesucristo vulcano verdugo de prometeo y creador de pand ora espiritu de dios juez de adan y eva pandora eva jupiter deidad superior a prometeo te del cuerpo de adan dios utiliza una par orden

Le decimos al modelo cómo debe responder basándonos en la unidad 6.

In [ ]:
def pregunta_y_contexto(prompt: str, context: str, api_key, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
                                    {
                                        "role": "system",
                                        "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos."
                                    },
                                    {
                                        "role": "user",
                                        "content": f"La información de contexto es la siguiente:\n\
                                        ---------------------\n\
                                        {context}\n\
                                        ---------------------\n\
                                        Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.\n\
                                        Pregunta: {prompt}\n\
                                        Respuesta: "
                                    }
                                ]

    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
#Ejemplo
respuesta_final = pregunta_y_contexto(prompt, contexto, api_key=api_key)

print(f'Prompt: {prompt}\n')
print('-----------------------------------------------------------------------')
print(f'Contexto: {contexto}\n')
print('-----------------------------------------------------------------------')
print(f'Respuesta:\n {respuesta_final}')

Prompt: ¿En qué fecha partió el RMS Titanic?

-----------------------------------------------------------------------
Contexto: que dan origen a la mi sma asi a vuelo de pajaro se ndria lo siguiente mitologia griega esis anti uo te libro del gen g testamento prometeo dios yahve prometeo poseedor del fuego del cielo dios poseedor del espiritu d vida e hombre de barr creado por prometeo adan creado por dios utilizando arcilla o prometeo roba el fuego del cielo dios concede el espiritu de vida a adan jupiter deidad superior a prometeo yahve divin idad suprema ordena al angel del senor ordena a vulcano encadenar a prometeo que adan y eva no vuelvan al ja eden rdin del pena temporal de cumplimiento efecti vo pena de eterno cumplimiento treinta anos acion de la pena ena hast a la mu te fisica duracion dur la vida terr er de la pena hercules liber tador de prometeo la muerte eterna mediante dios libera al humano de la promesa de jesucristo vulcano verdugo de prometeo y creador de pand ora esp

### Función que llame a las demás

In [ ]:
def chatbot(prompt, api_key):
    # Identificar la fuente
    fuente = elegir_fuente(prompt , api_key=api_key)

    # Formatear respuesta del LLM para la fuente
    fuente_limpia = formatear_respuesta(fuente)

    # Obtener el contexto de la fuente indicada y el ID de la fuente
    contexto, id_source = devolver_contexto(fuente_limpia, prompt)

    # Obtener la respuesta con el prompt original más el contexto obtenido
    respuesta_final = pregunta_y_contexto(prompt, contexto, api_key=api_key)

    return (respuesta_final, id_source, contexto)

## El chatbot en funcionamiento

In [ ]:
# Prompt:
prompt = '¿Quien es Jack Dawson?'

# Llamar a la función del chatbot
respuesta, fuente, contexto = chatbot(prompt, api_key)

# Imprimimos el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

Personaje: [Jack Dawson] en la película Titanic (1997), interpretado por el actor Leonardo DiCaprio. En la historia real del RMS Titanic, Jack Dawson fue un hombre real que viajó en el transatlántico en su último viaje, pero su destino diferente al de la película hace que se desconozcan sus identidad y suerte en el desastre.

En cuanto a la última pregunta, el personaje de Jack Dawson es un elemento ficticio de la película Titanic, creado por el guionista James Cameron para protagonizar la historia de amor entre él y el personaje de Rose DeWitt Bukater (interpretada por Kate Winslet). En la historia real del RMS Titanic, la identidad y el destino de cualquier pasajero o tripulante llamado Jack Dawson siguen siendo desconocidos.
------------------------------------------------------------------------
PREGUNTA:
¿Quien es Jack Dawson?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos de grafos
------------------------------------------

In [ ]:
# Prompt:
prompt = '¿Desde qué puerto partió el RMS Titanic?'

# Llamar a la función del chatbot
respuesta, fuente, contexto = chatbot(prompt, api_key)

# Imprimimos el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

Historia: El RMS Titanic zarpó desde el puerto de Southampton, Reino Unido, el 10 de abril de 1912, a las 2:30 p.m., marcando el comienzo de su trágica y legendaria travesía.
------------------------------------------------------------------------
PREGUNTA:
¿Desde qué puerto partió el RMS Titanic?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial de Embeddings
------------------------------------------------------------------------
CONTEXTO BRINDADO:
que dan origen a la mi sma asi a vuelo de pajaro se ndria lo siguiente mitologia griega esis anti uo te libro del gen g testamento prometeo dios yahve prometeo poseedor del fuego del cielo dios poseedor del espiritu d vida e hombre de barr creado por prometeo adan creado por dios utilizando arcilla o prometeo roba el fuego del cielo dios concede el espiritu de vida a adan jupiter deidad superior a prometeo yahve divin idad suprema ordena al angel del senor ordena a vulcano enc

In [ ]:
# Prompt:
prompt = '¿Cuántos sobrevivieron el choque?'

# Llamar a la función del chatbot
respuesta, fuente, contexto = chatbot(prompt, api_key)

# Imprimimos el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

Se han truncado las últimas 5000 líneas del flujo de salida.
Precio: 7.0
Cabina: nan
Embarco: S

PassengerId: 894
Sobrevivio: no
Clase: 2
Nombre: Myles, Mr. Thomas Francis
Sexo: male
Edad: 62.0
Parch: 0
Boleto: 240276
Precio: 9.6875
Cabina: nan
Embarco: Q

PassengerId: 895
Sobrevivio: no
Clase: 3
Nombre: Wirz, Mr. Albert
Sexo: male
Edad: 27.0
Parch: 0
Boleto: 315154
Precio: 8.6625
Cabina: nan
Embarco: S

PassengerId: 896
Sobrevivio: si
Clase: 3
Nombre: Hirvonen, Mrs. Alexander (Helga E Lindqvist)
Sexo: female
Edad: 22.0
Parch: 1
Boleto: 3101298
Precio: 12.2875
Cabina: nan
Embarco: S

PassengerId: 897
Sobrevivio: no
Clase: 3
Nombre: Svensson, Mr. Johan Cervin
Sexo: male
Edad: 14.0
Parch: 0
Boleto: 7538
Precio: 9.225
Cabina: nan
Embarco: S

PassengerId: 898
Sobrevivio: si
Clase: 3
Nombre: Connolly, Miss. Kate
Sexo: female
Edad: 30.0
Parch: 0
Boleto: 330972
Precio: 7.6292
Cabina: nan
Embarco: Q

PassengerId: 899
Sobrevivio: no
Clase: 2
Nombre: Caldwell, Mr. Albert Francis
Sexo: male
Edad: